In [1]:
import torch
import torch.nn.functional as F
torch.set_printoptions(profile='full')

In [11]:
zeros = torch.Tensor([[1,0,0]])
print(zeros.any())


tensor(True)


In [17]:
proto = torch.Tensor([[1,2,3]])
print(proto.size())
print(proto.squeeze(0).shape)
Z_p = torch.Tensor([[4,5,6], [7,8,9]])
Z_c = torch.Tensor([[10,11,12], [13,14,15]])
delta = Z_c - Z_p
print(delta)
dist = torch.sqrt(torch.sum(torch.pow(Z_p - proto, 2), dim=1))
print(dist)
dist_mean = torch.mean(dist)
print(dist_mean)
dist_variance = torch.mean(torch.pow(dist - dist_mean, 2))
print(dist_variance)
w = torch.exp(-torch.pow(dist, 2) / (2 * dist_variance))
print(w)
W = torch.sum(w)
print(W)
torch.sum(torch.unsqueeze(w, 1) * delta, dim=0) / W

torch.Size([1, 3])
torch.Size([3])
tensor([[6., 6., 6.],
        [6., 6., 6.]])
tensor([ 5.1962, 10.3923])
tensor(7.7942)
tensor(6.7500)
tensor([0.1353, 0.0003])
tensor(0.1357)


tensor([6., 6., 6.])

In [11]:
embedding = torch.randn(5)
F.normalize(embedding, dim=0)

tensor([ 0.1222, -0.7646,  0.0968, -0.6236, -0.0476])

In [12]:
batch = F.normalize(torch.randn(10,5), dim=1)
batch

tensor([[ 0.4342,  0.1807,  0.7055, -0.2508,  0.4671],
        [ 0.0895,  0.2692,  0.6800,  0.0687, -0.6727],
        [ 0.3958,  0.6374,  0.3359,  0.5280,  0.2135],
        [ 0.0264,  0.5472, -0.1960,  0.5855,  0.5645],
        [-0.2456, -0.0579,  0.2585,  0.9231,  0.1318],
        [ 0.2307, -0.3902,  0.5729,  0.6746,  0.1057],
        [-0.2358,  0.8479,  0.4216, -0.2022, -0.0823],
        [-0.3012, -0.5614,  0.5359,  0.5525, -0.0403],
        [-0.1251,  0.4225,  0.1585, -0.4812, -0.7410],
        [-0.4173, -0.0202, -0.2262,  0.8197,  0.3199]])

In [13]:
labels = torch.Tensor([0,1,1,0,0,2,2,0,3,3])

In [14]:
temperature = 0.01

In [15]:
dot_product_tempered = torch.mm(batch, batch.T) / temperature  # z_i \dot z_j / \tau
dot_product_tempered.size()  # [dim, dim](10,10)
dot_product_tempered

tensor([[100.0000,  23.5807,  49.1227,   8.8848, -10.4697,  31.3988,  36.0607,
          -1.1511,  -9.1560, -40.0674],
        [ 23.5807, 100.0000,  32.8078, -32.3088,  11.3072,  28.0451,  53.5296,
          25.1394,  67.5686, -35.5444],
        [ 49.1227,  32.8078, 100.0000,  72.3005,  46.8280,  41.3767,  46.4409,
          -1.3873, -13.9244,  24.7064],
        [  8.8848, -32.3088,  72.3005, 100.0000,  52.6008,  13.4899,  21.0263,
         -11.9435, -50.3236,  68.2788],
        [-10.4697,  11.3072,  46.8280,  52.6008, 100.0000,  75.0730,  -7.9680,
          74.9753, -49.4615,  84.3981],
        [ 31.3988,  28.0451,  41.3767,  13.4899,  75.0730, 100.0000, -28.8826,
          82.5112, -50.5886,  36.8759],
        [ 36.0607,  53.5296,  46.4409,  21.0263,  -7.9680, -28.8826, 100.0000,
         -28.7466,  61.2868, -20.6173],
        [ -1.1511,  25.1394,  -1.3873, -11.9435,  74.9753,  82.5112, -28.7466,
         100.0000, -35.0636,  45.5790],
        [ -9.1560,  67.5686, -13.9244, -50.3236,

In [16]:
max_ele = torch.max(dot_product_tempered, dim=1, keepdim=True)
print(max_ele)
dot_product_tempered = dot_product_tempered - max_ele[0]
print(dot_product_tempered)
exp_dot_tempered = (
    torch.exp(dot_product_tempered) 
    + 1e-5
)  # exp(z_i \dot z_j / \tau)，每一行代表和每个其他嵌入的点积
exp_dot_tempered

torch.return_types.max(
values=tensor([[100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000]]),
indices=tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]]))
tensor([[   0.0000,  -76.4193,  -50.8773,  -91.1152, -110.4697,  -68.6012,
          -63.9392, -101.1510, -109.1560, -140.0674],
        [ -76.4194,    0.0000,  -67.1922, -132.3088,  -88.6928,  -71.9549,
          -46.4704,  -74.8606,  -32.4314, -135.5444],
        [ -50.8773,  -67.1922,    0.0000,  -27.6995,  -53.1720,  -58.6233,
          -53.5591, -101.3873, -113.9244,  -75.2936],
        [ -91.1152, -132.3087,  -27.6995,    0.0000,  -47.3992,  -86.5101,
          -78.9737, -111.9435, -150.3236,  -31.7212],
        [-110.4697,  -88.6928,  -53.1720,  -47.3992,    0.0000,  -24.9270,
         -107.9680,  -25.0247, -149.

tensor([[1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0168e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00,
         1.0000e-05, 1.0025e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000

In [17]:
# 分母是z_i与batch中其他嵌入的点积，因此要屏蔽掉exp_dot_tempered中自己与自己相乘的部分，即对角线上的取值
unsqeezed_labels = labels.unsqueeze(1)  # [[0],[1],[1],[0],[0],[2],[2],[0],[3],[3]]
repeated_labels = unsqeezed_labels.repeat(1, labels.shape[0])  # 每一行都是batch_size长的全为对应的样本的标签的向量
mask_similar_class = (repeated_labels == labels)
print(mask_similar_class)
mask_anchor_out = (1 - torch.eye(exp_dot_tempered.shape[0]))  # 对角线为0其他全为1的batch_size * batch_size的张量
print(mask_anchor_out)

tensor([[ True, False, False,  True,  True, False, False,  True, False, False],
        [False,  True,  True, False, False, False, False, False, False, False],
        [False,  True,  True, False, False, False, False, False, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [False, False, False, False, False,  True,  True, False, False, False],
        [False, False, False, False, False,  True,  True, False, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True]])
tensor([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],

In [18]:
mask_combined = mask_similar_class * mask_anchor_out
mask_combined  # 每个样本在batch中相同样本的其他样本对应的位置为1，不同的为0

tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [19]:
cardinality_per_samples = torch.sum(mask_combined, dim=1)
cardinality_per_samples  # batch中每个样本的与他标签相同的其他样本的数量

tensor([3., 1., 1., 3., 3., 1., 1., 3., 1., 1.])

In [46]:
fenmu = torch.sum(exp_dot_tempered * mask_anchor_out, dim=1, keepdim=True)
print(fenmu)
log_prob = -torch.log(exp_dot_tempered / fenmu)
print(log_prob)

tensor([[9.0289e-05],
        [1.3500e-04],
        [1.3500e-04],
        [9.0001e-05],
        [9.0289e-05],
        [9.0000e-05],
        [9.0001e-05],
        [9.0000e-05],
        [9.0000e-05],
        [9.0000e-05]])
tensor([[-9.3125,  2.2004,  2.2004,  2.2004,  2.1720,  2.2004,  2.2004,  2.2004,
          2.2004,  2.2004],
        [ 2.6027, -8.9102,  0.8979,  2.6027,  2.6027,  2.6027,  2.6027,  2.6027,
          2.6027,  2.6027],
        [ 2.6027,  0.8979, -8.9102,  2.6027,  2.6027,  2.6027,  2.6027,  2.6027,
          2.6027,  2.6027],
        [ 2.1972,  2.1972,  2.1972, -9.3157,  2.1972,  2.1972,  2.1972,  2.1972,
          2.1972,  2.1972],
        [ 2.1720,  2.2004,  2.2004,  2.2004, -9.3125,  2.2004,  2.2004,  2.2004,
          2.2004,  2.2004],
        [ 2.1972,  2.1972,  2.1972,  2.1972,  2.1972, -9.3157,  2.1972,  2.1972,
          2.1972,  2.1972],
        [ 2.1972,  2.1972,  2.1972,  2.1972,  2.1972,  2.1972, -9.3157,  2.1972,
          2.1972,  2.1972],
        [ 2.1972

In [47]:
log = log_prob * mask_combined
print(log)
supervised_contrastive_loss_per_sample = torch.sum(log, dim=1) / cardinality_per_samples
print(supervised_contrastive_loss_per_sample)
supervised_contrastive_loss = torch.sum(supervised_contrastive_loss_per_sample)
print(supervised_contrastive_loss)


tensor([[-0.0000, 0.0000, 0.0000, 2.2004, 2.1720, 0.0000, 0.0000, 2.2004, 0.0000,
         0.0000],
        [0.0000, -0.0000, 0.8979, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.8979, -0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [2.1972, 0.0000, 0.0000, -0.0000, 2.1972, 0.0000, 0.0000, 2.1972, 0.0000,
         0.0000],
        [2.1720, 0.0000, 0.0000, 2.2004, -0.0000, 0.0000, 0.0000, 2.2004, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0000, 2.1972, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972, -0.0000, 0.0000, 0.0000,
         0.0000],
        [2.1972, 0.0000, 0.0000, 2.1972, 2.1972, 0.0000, 0.0000, -0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0000,
         2.1972],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972,
         -0.0000]])